In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
# biblioteka do rozpoznawania języka
from langdetect import detect, LangDetectException

- scrapping danych przy użyciu BeautifulSoup dla filmów Blade Runner 2049 oraz Drive (2011)
- wybranie recenzji tylko w języku angielskim
- usunięcie recenzji "This review may contain spoilers. I can handle the truth."

- jak opinie o ryanie goslingu i jego rosnąca popularność w branży fimowej oraz w sieci (przez żarty/memy typu sigma male) wpływała na recenzje na letterboxd? czy miały one bardziej pozytywny wydźwięk? Czy po powstaniu memu "literally me" (zdobywanie popularnościu memu 2022) widać większe zagęszczenie słów "literally", "sigma"? Czy samo nazwisko Gosling jest teraz częściej wspominane?

- czy wydźwięk recenzji pozytywny/negatywny zgadza się z oceną gwiazdkową? - porównanie różnych klasyfikatorów, wybranie najlepszego

najstarsze dane w datasecie są z 2011 (rok powstania Letterboxd), najnowsze z czerwca 2024

In [2]:
# https://letterboxd.com/film/in-the-mood-for-love/reviews/  - najnowsze
# https://letterboxd.com/film/in-the-mood-for-love/reviews/by/added-earliest/ - najstarsze
# https://letterboxd.com/film/in-the-mood-for-love/reviews/by/activity/ - najpopularniejsze
# https://letterboxd.com/film/in-the-mood-for-love/reviews/by/entry-rating/ - najlepiej oceniane
# https://letterboxd.com/film/in-the-mood-for-love/reviews/by/entry-rating-lowest/ - najgorzej oceniane

In [2]:
def convert_date(date_str):
    # Convert date from format '13 Jun 2024' to 'YYYY-MM-DD'
    try:
        date_obj = datetime.strptime(date_str, "%d %b %Y")
        if date_obj.year > 2500:   # niektore lata sa wyzsze niz obecna data, kalendarz buddyjski, 543 lata do przodu
            date_obj = date_obj.replace(year=date_obj.year - 543)
        return date_obj.strftime("%Y-%m-%d")
    except ValueError:
        return date_str  # Return the original string if there's an error in conversion

In [3]:
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False

In [39]:
def scrape_reviews(movie_title, limits=[1, 257]):
    all_reviews = []
    all_dates = []
    all_ratings = []

    endpoints = ["", "by/added-earliest/", "by/activity/", "by/entry-rating/", "by/entry-rating-lowest/"]
    base_url = f"https://letterboxd.com/film/{movie_title}/reviews/"
    for endpoint in endpoints:
        # jest dostępne maksymalnie 256 stron
        for i in range(limits[0], limits[1]):
            print(i, endpoint)
            url = base_url + endpoint
            if i != 1:
                url += f"page/{i}/"
            try:
                data = requests.get(url)
                data.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
                soup = BeautifulSoup(data.content, 'lxml')
                film_details = soup.find_all(class_="film-detail")

                for detail in film_details:
                    # Find review element
                    review_element = detail.find(class_="body-text -prose collapsible-text")
                    review_text = ""
                    if review_element:
                        review_text = review_element.find("p").text

                    # Check if the review is in English
                    if is_english(review_text):
                        all_reviews.append(review_text)

                        # Find date element
                        date_text = ""
                        date_element = detail.find(class_="_nobr")
                        if date_element:
                            date_text = date_element.text
                        all_dates.append(convert_date(date_text))  # Convert date here

                        # Find rating element
                        rating_text = ""
                        rating_element = detail.find(class_=lambda x: x and x.startswith('rating -green rated-'))
                        if rating_element:
                            # Extract the numerical rating from the class name
                            rating_class = rating_element['class']
                            for cls in rating_class:
                                if cls.startswith('rated-'):
                                    rating_text = cls.split('-')[-1]
                                    break
                        all_ratings.append(rating_text)
            except Exception as e:
                print(f"Error on page {i} with endpoint '{endpoint}': {e}")

    # Create DataFrame and save to CSV
    reviews_data = pd.DataFrame({
        'Date': all_dates,
        'Review': all_reviews,
        'Rating': all_ratings
    })

    csv_filename = f"{movie_title}_reviews.csv"
    reviews_data.to_csv(csv_filename, index=False, encoding='utf-8')
    print(f"Reviews, dates, and ratings have been saved to {csv_filename}")

In [40]:
# limits1= [[1, 31], [31, 61], [61, 91]]
# limits1 = []
# for i in range(0, 257//30):
#     if i == 0:
#         limits1.append([1, 31])
#     else:
#         limits1.append([limits1[i-1][1], limits1[i-1][1] + 30])

# limits1.append([241, 257])
# print(limits1)
# for limit in limits1:
scrape_reviews("american-psycho")

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 
56 
57 
58 
59 
60 
61 
62 
63 
64 
65 
66 
67 
68 
69 
70 
71 
72 
73 
74 
75 
76 
77 
78 
79 
80 
81 
82 
83 
84 
85 
86 
87 
88 
89 
90 
91 
92 
93 
94 
95 
96 
97 
98 
99 
100 
101 
102 
103 
104 
105 
106 
107 
108 
109 
110 
111 
112 
113 
114 
115 
116 
117 
118 
119 
120 
121 
122 
123 
124 
125 
126 
127 
128 
129 
130 
131 
132 
133 
134 
135 
136 
137 
138 
139 
140 
141 
142 
143 
144 
145 
146 
147 
148 
149 
150 
151 
152 
153 
154 
155 
156 
157 
158 
159 
160 
161 
162 
163 
164 
165 
166 
167 
168 
169 
170 
171 
172 
173 
174 
175 
176 
177 
178 
179 
180 
181 
182 
183 
184 
185 
186 
187 
188 
189 
190 
191 
192 
193 
194 
195 
196 
197 
198 
199 
200 
201 
202 
203 
204 
205 
206 
207 
208 
209 
210 
211 
212 
213 
214 
215 
216 
217 
218 
219 
220 
221 
222

In [41]:
scrape_reviews("fight-club")

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 
56 
57 
58 
59 
60 
61 
62 
63 
64 
65 
66 
67 
68 
69 
70 
71 
72 
73 
74 
75 
76 
77 
78 
79 
80 
81 
82 
83 
84 
85 
86 
87 
88 
89 
90 
91 
92 
93 
94 
95 
96 
97 
98 
99 
100 
101 
102 
103 
104 
105 
106 
107 
108 
109 
110 
111 
112 
113 
114 
115 
116 
117 
118 
119 
120 
121 
122 
123 
124 
125 
126 
127 
128 
129 
130 
131 
132 
133 
134 
135 
136 
137 
138 
139 
140 
141 
142 
143 
144 
145 
146 
147 
148 
149 
150 
151 
152 
153 
154 
155 
156 
157 
158 
159 
160 
161 
162 
163 
164 
165 
166 
167 
168 
169 
170 
171 
172 
173 
174 
175 
176 
177 
178 
179 
180 
181 
182 
183 
184 
185 
186 
187 
188 
189 
190 
191 
192 
193 
194 
195 
196 
197 
198 
199 
200 
201 
202 
203 
204 
205 
206 
207 
208 
209 
210 
211 
212 
213 
214 
215 
216 
217 
218 
219 
220 
221 
222

In [52]:

scrape_reviews("blade-runner-2049")

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 
56 
57 
58 
59 
60 
61 
62 
63 
64 
65 
66 
67 
68 
69 
70 
71 
72 
73 
74 
75 
76 
77 
78 
79 
80 
81 
82 
83 
84 
85 
86 
87 
88 
89 
90 
91 
92 
93 
94 
95 
96 
97 
98 
99 
100 
101 
102 
103 
104 
105 
106 
107 
108 
109 
110 
111 
112 
113 
114 
115 
116 
117 
118 
119 
120 
121 
122 
123 
124 
125 
126 
127 
128 
129 
130 
131 
132 
133 
134 
135 
136 
137 
138 
139 
140 
141 
142 
143 
144 
145 
146 
147 
148 
149 
150 
151 
152 
153 
154 
155 
156 
157 
158 
159 
160 
161 
162 
163 
164 
165 
166 
167 
168 
169 
170 
171 
172 
173 
174 
175 
176 
177 
178 
179 
180 
181 
182 
183 
184 
185 
186 
187 
188 
189 
190 
191 
192 
193 
194 
195 
196 
197 
198 
199 
200 
201 
202 
203 
204 
205 
206 
207 
208 
209 
210 
211 
212 
213 
214 
215 
216 
217 
218 
219 
220 
221 
222

In [53]:
scrape_reviews("drive-2011")

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 
13 
14 
15 
16 
17 
18 
19 
20 
21 
22 
23 
24 
25 
26 
27 
28 
29 
30 
31 
32 
33 
34 
35 
36 
37 
38 
39 
40 
41 
42 
43 
44 
45 
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 
56 
57 
58 
59 
60 
61 
62 
63 
64 
65 
66 
67 
68 
69 
70 
71 
72 
73 
74 
75 
76 
77 
78 
79 
80 
81 
82 
83 
84 
85 
86 
87 
88 
89 
90 
91 
92 
93 
94 
95 
96 
97 
98 
99 
100 
101 
102 
103 
104 
105 
106 
107 
108 
109 
110 
111 
112 
113 
114 
115 
116 
117 
118 
119 
120 
121 
122 
123 
124 
125 
126 
127 
128 
129 
130 
131 
132 
133 
134 
135 
136 
137 
138 
139 
140 
141 
142 
143 
144 
145 
146 
147 
148 
149 
150 
151 
152 
153 
154 
155 
156 
157 
158 
159 
160 
161 
162 
163 
164 
165 
166 
167 
168 
169 
170 
171 
172 
173 
174 
175 
176 
177 
178 
179 
180 
181 
182 
183 
184 
185 
186 
187 
188 
189 
190 
191 
192 
193 
194 
195 
196 
197 
198 
199 
200 
201 
202 
203 
204 
205 
206 
207 
208 
209 
210 
211 
212 
213 
214 
215 
216 
217 
218 
219 
220 
221 
222

In [ ]:
scrape_reviews("american-psycho")

In [ ]:
def preprocessing(title):
    df = pd.read_csv(f"{title}_reviews.csv")
    og_shape = df.shape[0]
    print(f"Liczba wierszy dla {title}:", df.shape[0])
    df = df[df['Review'] != "This review may contain spoilers. I can handle the truth."]
    df = df.drop_duplicates(subset=['Date', 'Review', 'Rating'])
    print(f"Liczba wierszy w {title} po usunięciu duplikatów i schowanych recenzji:", df.shape[0])
    df.to_csv(f"{title}_fixed.csv", index=False)
    return df, og_shape


In [ ]:
titles = ["blade-runner-2049", "drive-2011", "fight-club", "american-psycho"]
dfs = []
before = []
after = []
for name in titles:
    df, og_shape = preprocessing(name)
    dfs.append(df)
    before.append(og_shape)
    after.append(df.shape[0])

In [ ]:
print("titles:", titles)
print("before:", before, sum(before))
print("after:",  after, sum(after))
print(f"usunięto { (sum(before)-sum(after))/sum(before) * 100}% wierszy")

In [11]:
df_br = pd.read_csv("blade-runner-2049_reviews.csv")
df_d = pd.read_csv("drive-2011_reviews.csv")
og_shape = df_br.shape[0] + df_d.shape[0]


print("Liczba wierszy w blade-runner-2049_reviews.csv:", df_br.shape[0])
print("Liczba wierszy w drive-2011_reviews.csv:", df_d.shape[0])
print("razem:", df_br.shape[0] + df_d.shape[0])

# Usunięcie wierszy zawierających określony tekst w kolumnie 'Review'
df_br = df_br[df_br['Review'] != "This review may contain spoilers. I can handle the truth."]
df_d = df_d[df_d['Review'] != "This review may contain spoilers. I can handle the truth."]

print("\npo usunieciu wierszy ze schowanym Review")
print("Liczba wierszy w blade-runner-2049_reviews.csv:", df_br.shape[0])
print("Liczba wierszy w drive-2011_reviews.csv:", df_d.shape[0])
print("razem:", df_br.shape[0] + df_d.shape[0])

df_br = df_br.drop_duplicates(subset=['Date', 'Review', 'Rating'])
df_d = df_d.drop_duplicates(subset=['Date', 'Review', 'Rating'])
print("\npo usunieciu duplikatów")
print("Liczba wierszy w blade-runner-2049_reviews.csv:", df_br.shape[0])
print("Liczba wierszy w drive-2011_reviews.csv:", df_d.shape[0])
print("razem:", df_br.shape[0] + df_d.shape[0])

Liczba wierszy w blade-runner-2049_reviews.csv: 10737
Liczba wierszy w drive-2011_reviews.csv: 10626
razem: 21363

po usunieciu wierszy ze schowanym Review
Liczba wierszy w blade-runner-2049_reviews.csv: 10201
Liczba wierszy w drive-2011_reviews.csv: 10263
razem: 20464

po usunieciu duplikatów
Liczba wierszy w blade-runner-2049_reviews.csv: 9296
Liczba wierszy w drive-2011_reviews.csv: 9558
razem: 18854


In [12]:
print("usunięto", (og_shape - df_br.shape[0] - df_d.shape[0])/(og_shape) * 100, '% wierszów')

usunięto 11.744605158451527 % wierszów


In [13]:
df_br.to_csv("blade-runner-2049_reviews_fixed.csv", index=False)
df_d.to_csv("drive-2011_reviews_fixed.csv", index=False)
